# Triangle Counting

In this notebook, we will count the numner of trianges in our test dataset.  The NetworkX and cuGraph processes will be interleaved so that each step can be compared.

Notebook Credits
* Original Authors: Bradley Rees
* Last Edit: 08/09/2019

RAPIDS Versions: 0.9.0    

Test Hardware

* GV100 32G, CUDA 10.0


## Introduction
Triancle Counting, as the name implies, finds the number of triangles in a graph.  Triangles are important in computing the clustering Coefficient and can be used for clustering.  


To compute the Pagerank scores for a graph in cuGraph we use:<br>

**cugraph.triangles(G)**
* __G__: cugraph.Graph object


Returns:
* int64 count
    
<br><br>
## cuGraph 0.7 Notice 
cuGraph version 0.7 has some limitations:
* Only Int32 Vertex ID are supported
* Only float (FP32) edge data is supported
* Vertex numbering is assumed to start at zero

These limitations are being addressed and will be fixed future versions.  
These example notebooks will illustrate how to manipulate the data so that it comforms to the current limitations    

### Test Data
We will be using the Zachary Karate club dataset 
*W. W. Zachary, An information flow model for conflict and fission in small groups, Journal of
Anthropological Research 33, 452-473 (1977).*


![Karate Club](./img/zachary_black_lines.png)


### Prep

In [1]:
# The notebook compares cuGraph to NetworkX,  
# therefore there some additional non-RAPIDS python libraries need to be installed. 
# Please run this cell if you need the additional libraries
!pip install networkx
!pip install scipy


In [2]:
# Import needed libraries
import cugraph
import cudf
from collections import OrderedDict

In [3]:
# NetworkX libraries
import networkx as nx
from scipy.io import mmread

### Some Prep

In [4]:
# Define the path to the test data  
datafile='./data/karate-data.csv'

---
# NetworkX

In [5]:
# Read the data, this also created a NetworkX Graph 
file = open(datafile, 'rb')
Gnx = nx.read_edgelist(file)

In [6]:
nx_count = nx.triangles(Gnx)

In [7]:
# NetworkX does not give a single count, but list how many triangles each vertex is associated with
nx_count

{'1': 18,
 '2': 12,
 '3': 11,
 '4': 10,
 '5': 2,
 '6': 3,
 '7': 3,
 '8': 6,
 '9': 5,
 '11': 2,
 '12': 0,
 '13': 1,
 '14': 6,
 '18': 1,
 '20': 1,
 '22': 1,
 '32': 3,
 '31': 3,
 '10': 0,
 '28': 1,
 '29': 1,
 '33': 13,
 '17': 1,
 '34': 15,
 '15': 1,
 '16': 1,
 '19': 1,
 '21': 1,
 '23': 1,
 '24': 4,
 '26': 1,
 '30': 4,
 '25': 1,
 '27': 1}

In [8]:
# To get the number of triangles, we would need to loop through the array and add up each count
count = 0
for key, value in nx_count.items():
    count = count + value

In [9]:
count

135

Let's seet how that compares to cuGraph

----

# cuGraph

### Read in the data - GPU
cuGraph depends on cuDF for data loading and the initial Dataframe creation

The data file contains an edge list, which represents the connection of a vertex to another.  The `source` to `destination` pairs is in what is known as Coordinate Format (COO).  In this test case, the data is just two columns.  However a third, `weight`, column is also possible

In [10]:
# Test file    
datafile='./data/karate-data.csv'

# Read the data file
cols = ["src", "dst"]

dtypes = OrderedDict([
        ("src", "int32"), 
        ("dst", "int32")
        ])

gdf = cudf.read_csv(datafile, names=cols, delimiter='\t', dtype=list(dtypes.values()) )

### Create a Graph 

In [11]:
# create a Graph using the source (src) and destination (dst) vertex pairs from the Dataframe 
G = cugraph.Graph()
G.add_edge_list(gdf["src"], gdf["dst"])

### Call the Triangle Counting 

In [12]:
# Call cugraph.pagerank to get the pagerank scores
cu_count = cugraph.triangles(G)

In [13]:
cu_count

135

_It was that easy!_  

----


___
Copyright (c) 2019, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___